# Import pour le code en général

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from datetime import timedelta
import movingpandas as mpd
import hvplot.pandas
import json
import contextily as ctx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pykalman
from sqlalchemy import text
from sqlalchemy import create_engine

# 1. Load the data from the csv files

In [ ]:
# Charger les fichiers CSV
activity_df = pd.read_csv("Activity_self_report.csv", sep=';', parse_dates=['time'])
print("1. Données des activités chargées :\n", activity_df.head())

gps_log_df = pd.read_csv("GPS_log.csv", sep=';', parse_dates=['timestamp'])
print("2. Données GPS chargées :\n", gps_log_df.head())

sensor_measures_df = pd.read_csv("Sensor_Measures.csv", sep=';', parse_dates=['time'])
print("3. Données des capteurs chargées :\n", sensor_measures_df.head())


# 2. Segment the data per day

In [ ]:
# Supprimer les fuseaux horaires pour éviter les erreurs
gps_log_df['timestamp'] = gps_log_df['timestamp'].dt.tz_localize(None)
activity_df['time'] = activity_df['time'].dt.tz_localize(None)
sensor_measures_df['time'] = sensor_measures_df['time'].dt.tz_localize(None)
print("4. Fuseaux horaires supprimés des données")
print("2. Données GPS chargées :\n", gps_log_df.head())
print("2. Données activite chargées :\n", activity_df.head())
print("2. Données sensor chargées :\n", sensor_measures_df.head())

In [ ]:
# 🔹 Ajouter une colonne 'date' en extrayant uniquement YYYY-MM-DD
gps_log_df["date"] = gps_log_df["timestamp"].dt.date
activity_df["date"] = activity_df["time"].dt.date
sensor_measures_df["date"] = sensor_measures_df["time"].dt.date

# 🔹 Vérifier les premières valeurs
print("Données GPS segmentées par jour :\n", gps_log_df.head())
print("Données d'activités segmentées par jour :\n", activity_df.head())
print("Données capteurs segmentées par jour :\n", sensor_measures_df.head())

# 🔹 Grouper les données par jour (optionnel)
gps_grouped = gps_log_df.groupby("date")
activity_grouped = activity_df.groupby("date")
sensor_grouped = sensor_measures_df.groupby("date")

# 🔹 Afficher un aperçu des groupes
print(f"Nombre de jours distincts dans les données GPS : {len(gps_grouped)}")
print(f"Nombre de jours distincts dans les activités : {len(activity_grouped)}")
print(f"Nombre de jours distincts dans les capteurs : {len(sensor_grouped)}")

# 3. Transform the scalar data to spatial type in geopandas and spatiotemporal type in movingpandas and visualize them. You may create different versions to represent the data at each stage.

In [ ]:
# Create a GeoDataFrame 
gps_log_df['geometry'] = [Point(xy) for xy in zip(gps_log_df['lon'], gps_log_df['lat'])]
gdf = gpd.GeoDataFrame(gps_log_df, geometry='geometry')
gdf.set_crs(epsg=4326, inplace=True)  # Assuming the coordinates are in WGS84


In [ ]:
#Créer des points géométriques pour les données GPS
geometry = [Point(xy) for xy in zip(gps_log_df['lon'], gps_log_df['lat'])]
gdf = gpd.GeoDataFrame(gps_log_df, geometry=geometry, crs="EPSG:4326")

#Ajouter un identifiant de trajectoire basé sur l'utilisateur ou une logique arbitraire
if 'user_id' in gdf.columns:
    gdf['trajectory_id'] = gdf['user_id']
else:
    gdf['trajectory_id'] = gdf.index // 100  # Exemple : diviser en blocs arbitraires

#Transformation en TrajectoryCollection avec MovingPandas
trajectory_collection = mpd.TrajectoryCollection(gdf, traj_id_col='trajectory_id', t='timestamp')

#Validation des trajectoires créées
print(f"Nombre de trajectoires détectées : {len(trajectory_collection)}")
for traj in trajectory_collection:
    print(traj)

#Visualisation des trajectoires pour validation
map_visualization = gdf.hvplot(geo=True, tiles='OSM', c='trajectory_id', line_width=2, width=800, height=600)
map_visualization

In [ ]:
# Assume 'trajectory_collection' is already created and contains trajectories
for trajectory in trajectory_collection:
    print(trajectory)
    # Adding distance in default units (meters) and custom units (kilometers and yards)
    trajectory.add_distance(overwrite=True, name="distance_m", units="m")
    trajectory.add_distance(overwrite=True, name="distance_km", units="km")
    trajectory.add_distance(overwrite=True, name="distance_yards", units="yd")

    # Adding speed in default units (m/s) and custom units (feet/minute and knots)
    trajectory.add_speed(overwrite=True, name="speed_kmph", units=("km", "h"))
    trajectory.add_speed(overwrite=True, name="speed_ft_min", units=("ft", "min"))
    trajectory.add_speed(overwrite=True, name="speed_knots", units=("nm", "h"))

    # Correctly adding acceleration with appropriate time units
    trajectory.add_acceleration(overwrite=True, name="acceleration_mps2", units=("m", "s", "s"))
    trajectory.add_acceleration(overwrite=True, name="acceleration_mph_s", units=("mi", "h", "s"))


    # Print the updated dataframe to see the added columns
    print(trajectory.df.head())

#Visualisation des trajectoires pour validation
map_visualization = gdf.hvplot(geo=True, tiles='OSM', c='trajectory_id', line_width=2, width=800, height=600)
map_visualization


In [ ]:
# For interactive visualization with hvplot
import hvplot.pandas
hvplot_defaults = {
    "tiles": None,
    "frame_height": 320,
    "frame_width": 320,
    "cmap": 'Viridis',
    "colorbar": True
}
trajectory.hvplot(c='speed_kmph', **hvplot_defaults)

# 4. Clean the trajectories and the temporal measures and check the result visually

In [ ]:
# Nettoyage des trajectoires avec suppression des outliers
cleaned_trajectories = []
nb_trajectoires_initiales = len(trajectory_collection.trajectories)

for trajectory in trajectory_collection.trajectories:  
    cleaner = mpd.OutlierCleaner(trajectory)  
    cleaned_traj = cleaner.clean(alpha=2)  # Alpha définit la tolérance aux outliers
    
    # Vérifier si la trajectoire nettoyée contient encore des données
    if cleaned_traj is not None and not cleaned_traj.df.empty:
        cleaned_traj.add_speed(overwrite=True, units=("km", "h"))  # Recalculer la vitesse
        cleaned_trajectories.append(cleaned_traj)
        print(f"✅ Trajectoire {trajectory.id} nettoyée : {len(trajectory.df)} → {len(cleaned_traj.df)} points restants")
    else:
        print(f"❌ Trajectoire {trajectory.id} supprimée après nettoyage (trop d'outliers)")

# Vérification du nombre de trajectoires conservées
nb_trajectoires_nettoyees = len(cleaned_trajectories)
print(f"\n📌 Nombre de trajectoires avant nettoyage : {nb_trajectoires_initiales}")
print(f"📌 Nombre de trajectoires après nettoyage : {nb_trajectoires_nettoyees}")

# Création d'une nouvelle collection avec les trajectoires nettoyées
cleaned_trajectory_collection = mpd.TrajectoryCollection(cleaned_trajectories)

# Visualisation interactive des trajectoires nettoyées
gdf_cleaned = gpd.GeoDataFrame(pd.concat([t.df for t in cleaned_trajectories]), crs="EPSG:4326")
map_cleaned = gdf_cleaned.hvplot(
    geo=True, tiles='OSM', c='trajectory_id', line_width=2, width=800, height=600
)
map_cleaned

# Option : Visualisation statique avec une carte de fond

fig, ax = plt.subplots(figsize=(10, 6))
gdf_cleaned.plot(ax=ax, alpha=0.7, edgecolor='k')
ctx.add_basemap(ax, crs=gdf_cleaned.crs, source=ctx.providers.OpenStreetMap.Mapnik)
plt.title("Trajectoires après nettoyage des outliers")
plt.show()

In [ ]:
# Clean each trajectory by removing outliers
cleaned_trajectories = []
for trajectory in trajectory_collection:
    # Split into segments if needed and apply OutlierCleaner
    cleaner = mpd.OutlierCleaner(trajectory)
    cleaned_traj = cleaner.clean(alpha=2)  # Alpha defines how strict the threshold is
    cleaned_traj.add_speed(overwrite=True, units=("km", "h"))  # Re-add speed for visualization
    cleaned_trajectories.append(cleaned_traj)
    print(f"Cleaned Trajectory for User {trajectory.id}")
    print(cleaned_traj.df.head())

In [ ]:
# Visualize after cleaning
import hvplot.pandas  # Required for interactive visualization

hvplot_defaults = {
    "tiles": "CartoLight",
    "frame_height": 400,
    "frame_width": 500,
    "cmap": "Viridis",
    "colorbar": True,
}

# Example: Plot the cleaned trajectory for the first user
trajectory_collection.trajectories[0].hvplot(
    label="Before Cleaning", color="red", line_width=4, **hvplot_defaults
) * cleaned_trajectories[0].hvplot(
    label="After Cleaning", c="speed", line_width=4
)

# 5. Detect the stops (visits) and the moves (travels)

In [ ]:
# Créer une collection de trajectoires
tc = mpd.TrajectoryCollection(cleaned_trajectories, 'user_id', t='time')

# Initialiser le détecteur de stops
detector = mpd.TrajectoryStopDetector(tc)

# Détecter les arrêts (Stops)
stop_segments = detector.get_stop_segments(min_duration=timedelta(minutes=7), max_diameter=100)

# Créer une liste pour stocker les moves
move_segments = []

# Identifier les moves comme étant les périodes entre les stops
for traj in tc.trajectories:
    stop_times = [(stop.df.index.min(), stop.df.index.max()) for stop in stop_segments]

    # Trier les stops pour éviter les erreurs
    stop_times.sort()

    prev_end = traj.df.index.min()  # Début de la trajectoire

    for start, end in stop_times:
        # Extraire les moves entre la fin du stop précédent et le début du stop actuel
        move_df = traj.df[(traj.df.index > prev_end) & (traj.df.index < start)]
        
        # Vérifier qu'il y a au moins 2 points avant d'ajouter
        if len(move_df) >= 2:
            move_segments.append(mpd.Trajectory(move_df, traj.id))

        prev_end = end  # Mettre à jour pour le prochain move

# Vérification des résultats
print(f"✅ Nombre de stops détectés : {len(stop_segments)}")
print(f"✅ Nombre de moves détectés : {len(move_segments)}")

# Convertir les résultats en GeoDataFrames
stop_gdf = stop_segments.to_point_gdf()
move_gdf = gpd.GeoDataFrame(pd.concat([traj.to_line_gdf() for traj in move_segments]), crs="EPSG:4326")

# Afficher les premiers résultats
print("✅ Stops détectés :")
print(stop_gdf.head())

print("\n✅ Moves détectés :")
print(move_gdf.head())

# Sauvegarder les résultats dans des fichiers CSV
stop_gdf.to_csv("Detected_Stops.csv", index=False)
move_gdf.to_csv("Detected_Moves.csv", index=False)

print("\n📂 Résultats exportés : 'Detected_Stops.csv' et 'Detected_Moves.csv'")

# 6. Segment the data based on the stops and moves ; propagate the segmentation to the measurement part.

In [ ]:
# Charger les mesures capteurs
sensor_measures_df = pd.read_csv("Sensor_Measures.csv", sep=';', parse_dates=['time'])

# Trier les mesures pour éviter les erreurs de correspondance
sensor_measures_df = sensor_measures_df.sort_values(by="time")

# **Correction : Supprimer les fuseaux horaires**
sensor_measures_df['time'] = sensor_measures_df['time'].dt.tz_localize(None)

# Appliquer la correction sur les stops et moves
for stop in stop_segments:
    stop.df.index = stop.df.index.tz_localize(None)

for move in move_segments:
    move.df.index = move.df.index.tz_localize(None)

# Ajouter une colonne 'Segment_Type' aux mesures capteurs
sensor_measures_df['Segment_Type'] = np.nan
sensor_measures_df['Segment_ID'] = np.nan  # Ajout de l'ID du segment pour analyse

# Associer chaque mesure capteur au segment le plus proche
for stop in stop_segments:
    mask = (sensor_measures_df['time'] >= stop.df.index.min()) & (sensor_measures_df['time'] <= stop.df.index.max())
    sensor_measures_df.loc[mask, 'Segment_Type'] = 'Stop'
    sensor_measures_df.loc[mask, 'Segment_ID'] = stop.id  # Associer l'ID du segment Stop

for move in move_segments:
    mask = (sensor_measures_df['time'] >= move.df.index.min()) & (sensor_measures_df['time'] <= move.df.index.max())
    sensor_measures_df.loc[mask, 'Segment_Type'] = 'Move'
    sensor_measures_df.loc[mask, 'Segment_ID'] = move.id  # Associer l'ID du segment Move

# Supprimer les mesures sans segment associé
sensor_measures_df.dropna(subset=['Segment_Type'], inplace=True)

# Afficher les premiers résultats
print(sensor_measures_df.head())

# Sauvegarder le fichier segmenté
sensor_measures_df.to_csv("Segmented_Sensor_Data.csv", index=False)
print("📂 Données segmentées exportées : 'Segmented_Sensor_Data.csv'")

In [ ]:
# Plot stops
hvplot_defaults = {
    "tiles": "CartoLight",
    "frame_height": 400,
    "frame_width": 600,
    "cmap": "Viridis",
    "colorbar": True
}
tc_plot = tc.hvplot(color="slategray", **hvplot_defaults)
stop_segment_plot = stop_segments.hvplot(line_width=7.0, color="deeppink")
(tc_plot * stop_segment_plot).opts(title="Stops and Moves Visualization")


# 7. Does the result conform to the self-reported changes? Discuss the gaps; try different parameters in the preprocessing, such as the stop duration, smoothing trajectories beforehand, etc.

In [ ]:
# Définition des catégories d'activités
stops = {"Bureau", "Domicile", "Magasin", "Parc"}
moves = {"Rue", "Bus", "Train", "Métro"}

# Ajouter une colonne 'Type' pour classer les activités
activity_df['Type'] = activity_df['activity'].apply(lambda x: "Stop" if x in stops else "Move" if x in moves else "Unknown")

# Calculer la durée de chaque activité (différence entre les timestamps)
activity_df['duration'] = activity_df['time'].diff().shift(-1)

# Afficher un aperçu
print(activity_df)

# Visualisation du temps passé en "Stop" vs "Move"
summary = activity_df.groupby("Type")["duration"].sum()
summary_hours = summary.dt.total_seconds() / 3600  # Convertir en heures

# Affichage sous forme de diagramme
plt.figure(figsize=(6, 4))
summary_hours.plot(kind='bar', color=['green', 'red'])
plt.ylabel("Temps total (heures)")
plt.title("Temps passé en 'Stop' et 'Move'")
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Définir un ID de segment unique pour chaque période de Stop ou Move
activity_df['Segment_ID'] = (activity_df['Type'] != activity_df['Type'].shift()).cumsum()

# Fusionner avec le DataFrame des mesures pour propager la segmentation
sensor_measures_df = sensor_measures_df.sort_values(by="time")  # Assurer l'ordre chronologique

# Associer chaque mesure au bon segment (on prend le segment en cours au moment de la mesure)
sensor_measures_df['Segment_ID'] = np.nan  # Initialisation

for i, row in activity_df.iterrows():
    # Trouver les mesures qui se situent entre cette activité et la suivante
    if i < len(activity_df) - 1:
        mask = (sensor_measures_df['time'] >= row['time']) & (sensor_measures_df['time'] < activity_df.loc[i+1, 'time'])
    else:
        mask = sensor_measures_df['time'] >= row['time']
    
    # Associer le Segment_ID et le Type
    sensor_measures_df.loc[mask, 'Segment_ID'] = row['Segment_ID']
    sensor_measures_df.loc[mask, 'Type'] = row['Type']

# Nettoyage final
sensor_measures_df.dropna(subset=['Segment_ID'], inplace=True)  # Supprimer les mesures sans segment associé

# Afficher un aperçu des mesures segmentées
pd.set_option('display.max_columns', None)
print(sensor_measures_df)

In [ ]:
# Trier les données pour un bon alignement temporel
sensor_measures_df = sensor_measures_df.sort_values(by="time")
activity_df = activity_df.sort_values(by="time")

# Comparer en utilisant une fusion approximative (tolérance de 30 secondes)
comparison_df = pd.merge_asof(
    sensor_measures_df[['time', 'Segment_Type']],  # Automatique
    activity_df[['time', 'Type']],  # Manuelle
    on='time',
    direction='nearest',  # Prend la valeur la plus proche
    tolerance=pd.Timedelta(seconds=30)  # Tolérance de 30 sec
)

# Renommer les colonnes pour plus de clarté
comparison_df.rename(columns={'Segment_Type': 'Type_detected', 'Type': 'Type_reported'}, inplace=True)

# Supprimer les lignes où il n'y a pas d'auto-déclaration (évite les NaN)
comparison_df.dropna(subset=['Type_reported'], inplace=True)

# Détecter les incohérences
comparison_df['Mismatch'] = comparison_df['Type_detected'] != comparison_df['Type_reported']

# Compter les erreurs
nb_mismatches = comparison_df['Mismatch'].sum()
total_entries = len(comparison_df)

print(f"Nombre total de différences entre les données détectées et auto-déclarées : {nb_mismatches} / {total_entries}")
print(f"Erreur de détection : {round((nb_mismatches / total_entries) * 100, 2)}%")

# Afficher les cas problématiques
print("\nExemples d'incohérences détectées :")
print(comparison_df[comparison_df['Mismatch']].head(10))

# Table de confusion entre les catégories
labels = ["Stop", "Move"]
cm = confusion_matrix(comparison_df["Type_reported"], comparison_df["Type_detected"], labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

# Affichage de la table de confusion
fig, ax = plt.subplots(figsize=(6, 4))
disp.plot(ax=ax, cmap="Blues")
plt.title("Table de confusion : Détections vs Auto-déclarations")
plt.show()

# Visualisation des erreurs avec un diagramme à barres
plt.figure(figsize=(6, 4))
comparison_df['Mismatch'].value_counts().plot(kind='bar', color=['green', 'red'])
plt.xticks([0, 1], ['Correspondance', 'Incohérence'], rotation=0)
plt.ylabel("Nombre de cas")
plt.title("Correspondance entre auto-déclarations et détections")
plt.show()

In [ ]:
# Charger les données GPS
gps_log_df = pd.read_csv("GPS_log.csv", sep=';', parse_dates=['timestamp'])

# Suppression des valeurs nulles éventuelles
gps_log_df = gps_log_df.dropna(subset=['lat', 'lon'])

# Appliquer un filtre de Kalman pour lisser les coordonnées GPS
kalman_filter = pykalman.KalmanFilter(
    initial_state_mean=[gps_log_df['lat'].iloc[0], gps_log_df['lon'].iloc[0]],
    transition_matrices=np.eye(2),
    observation_matrices=np.eye(2),
    initial_state_covariance=1e-4 * np.eye(2),
    observation_covariance=1e-1 * np.eye(2),
    transition_covariance=1e-4 * np.eye(2)
)

# Exécuter le filtre sur les coordonnées
filtered_state_means, _ = kalman_filter.filter(gps_log_df[['lat', 'lon']].values)

# Ajouter les nouvelles coordonnées lissées dans le DataFrame
gps_log_df['lat_smooth'] = filtered_state_means[:, 0]
gps_log_df['lon_smooth'] = filtered_state_means[:, 1]

# Créer des GeoDataFrames pour visualisation
gps_log_df['geometry'] = [Point(xy) for xy in zip(gps_log_df['lon'], gps_log_df['lat'])]
gps_log_df['geometry_smooth'] = [Point(xy) for xy in zip(gps_log_df['lon_smooth'], gps_log_df['lat_smooth'])]

gdf = gpd.GeoDataFrame(gps_log_df, geometry='geometry', crs="EPSG:4326")
gdf_smooth = gpd.GeoDataFrame(gps_log_df, geometry='geometry_smooth', crs="EPSG:4326")

# Visualisation avant/après filtrage
fig, ax = plt.subplots(figsize=(10, 6))
gdf.plot(ax=ax, alpha=0.5, label="Trajectoire brute", color="red")
gdf_smooth.plot(ax=ax, alpha=0.8, label="Trajectoire lissée", color="blue")
plt.legend()
plt.title("Comparaison des trajectoires brutes et lissées (Filtre de Kalman)")
plt.show()

# 8. Use movingpandas to export the trajectories (each stop and move segments) in MF-JSON format.


In [ ]:
# Convertir les stops en Trajectoires
stop_trajectories = [mpd.Trajectory(stop.df, stop.id) for stop in stop_segments if len(stop.df) > 1]

# Convertir les moves en Trajectoires
move_trajectories = [mpd.Trajectory(move.df, move.id) for move in move_segments if len(move.df) > 1]

# Créer une collection contenant les Stops
stop_collection = mpd.TrajectoryCollection(stop_trajectories)

# Créer une collection contenant les Moves
move_collection = mpd.TrajectoryCollection(move_trajectories)

# Convertir les collections en GeoDataFrame
gdf_stops = gpd.GeoDataFrame(pd.concat([traj.to_line_gdf() for traj in stop_collection.trajectories]), crs="EPSG:4326")
gdf_moves = gpd.GeoDataFrame(pd.concat([traj.to_line_gdf() for traj in move_collection.trajectories]), crs="EPSG:4326")

# Exporter en GeoJSON (compatible MF-JSON)
stop_filename = "stops_segmented.geojson"
move_filename = "moves_segmented.geojson"

gdf_stops.to_file(stop_filename, driver="GeoJSON")
gdf_moves.to_file(move_filename, driver="GeoJSON")

print(f"✅ Stops exportés en MF-JSON : {stop_filename}")
print(f"✅ Moves exportés en MF-JSON : {move_filename}")

# 9. Load the pre-processed data into MobilityDB using the MF-JSON object obtained previously as input.

In [ ]:
# Configuration de la connexion PostgreSQL (MobilityDB dans Docker)
DB_NAME = "mobilitydb"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"  # Ou l'IP de ton conteneur
DB_PORT = "5433"  # Assure-toi que c'est bien le port exposé

# Connexion à PostgreSQL
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

print("✅ Connexion à MobilityDB réussie !")

In [ ]:
# Charger les fichiers GeoJSON
stops_gdf = gpd.read_file("stops_segmented.geojson")
moves_gdf = gpd.read_file("moves_segmented.geojson")

# Ajouter une colonne pour identifier Stop ou Move
stops_gdf["segment_type"] = "Stop"
moves_gdf["segment_type"] = "Move"

# Fusionner les deux DataFrames
full_gdf = pd.concat([stops_gdf, moves_gdf])

print(f"📊 Nombre de segments chargés : {len(full_gdf)}")

In [ ]:
# Nom de la table dans MobilityDB
TABLE_NAME = "mobility_trajectories"

# Importer les données dans PostgreSQL
full_gdf.to_postgis(TABLE_NAME, engine, if_exists="replace", index=False)

print(f"✅ Données importées dans la table {TABLE_NAME} de MobilityDB")

In [ ]:
# Vérifier la structure de la table
with engine.connect() as conn:
    result = conn.execute(text("SELECT column_name FROM information_schema.columns WHERE table_name = 'mobility_trajectories';"))
    columns = [row[0] for row in result.fetchall()]
    print("📋 Colonnes existantes :", columns)

# 10. For each stop segment, assign a centroid location, its time range and its duration, as well as the maximum distance in meters to the bounding box (this reflects the uncertainty of the location).

In [ ]:
# Vérifier les colonnes disponibles dans la table mobility_trajectories
query = text("""
    SELECT column_name 
    FROM information_schema.columns 
    WHERE table_name = 'mobility_trajectories';
""")

# Exécuter la requête et récupérer les colonnes
with engine.connect() as conn:
    result = conn.execute(query)
    columns = [row[0] for row in result.fetchall()]

# Afficher les colonnes disponibles pour adapter la requête
print("📋 Colonnes disponibles dans 'mobility_trajectories':", columns)

In [ ]:
# Requête SQL corrigée pour MobilityDB
query = text("""
    WITH StopAggregates AS (
        SELECT 
            traj_id,
            ST_Centroid(ST_Collect(geometry)) AS centroid, 
            MIN(t) AS start_time, 
            MAX(t) AS end_time, 
            (MAX(t) - MIN(t)) AS duration
        FROM mobility_trajectories 
        WHERE segment_type = 'Stop'
        GROUP BY traj_id
    )
    SELECT 
        s.traj_id,
        s.centroid,
        s.start_time,
        s.end_time,
        s.duration,
        MAX(ST_Distance(m.geometry, s.centroid)) AS max_distance_bbox 
    FROM mobility_trajectories m
    JOIN StopAggregates s ON m.traj_id = s.traj_id
    WHERE m.segment_type = 'Stop'
    GROUP BY s.traj_id, s.centroid, s.start_time, s.end_time, s.duration;
""")

# Exécuter la requête et charger les résultats dans un DataFrame
with engine.connect() as conn:
    stop_summary = pd.read_sql(query, conn)

# Sauvegarder les résultats dans un fichier CSV
output_path = "Stop_Summary.csv"
stop_summary.to_csv(output_path, index=False)

# Afficher un message de confirmation
print(f"✅ Résumé des Stops enregistré dans '{output_path}'.")

# 11. Find recurrent stops:matchthe stop segments based on the distance between the centroids and the intersection of their box, and assign them the same tag or stop_id.

In [ ]:
# Requête SQL pour détecter les Stops récurrents
query = text("""
    WITH StopAggregates AS (
        SELECT 
            traj_id,
            ST_Centroid(ST_Collect(geometry)) AS centroid,
            ST_Envelope(ST_Collect(geometry)) AS bbox,
            MIN(t) AS start_time,
            MAX(t) AS end_time,
            (MAX(t) - MIN(t)) AS duration
        FROM mobility_trajectories 
        WHERE segment_type = 'Stop'
        GROUP BY traj_id
    )
    SELECT 
        sa1.traj_id AS stop_id_1,
        sa2.traj_id AS stop_id_2,
        ST_Distance(sa1.centroid, sa2.centroid) AS centroid_distance,
        ST_Intersects(sa1.bbox, sa2.bbox) AS bbox_intersection
    FROM StopAggregates sa1
    JOIN StopAggregates sa2
    ON sa1.traj_id < sa2.traj_id
    WHERE ST_Distance(sa1.centroid, sa2.centroid) < 30
    AND ST_Intersects(sa1.bbox, sa2.bbox) = TRUE;
""")

# Exécuter la requête et charger les résultats dans un DataFrame
with engine.connect() as conn:
    recurrent_stops = pd.read_sql(query, conn)

# Sauvegarder les résultats dans un fichier CSV
output_path = "Recurrent_Stops.csv"
recurrent_stops.to_csv(output_path, index=False)

# Afficher un message de confirmation
print(f"✅ Détection des Stops récurrents terminée ! Résultats enregistrés dans '{output_path}'.")

# 12. Rank the stops by the frequency of visits(the most visited first).Rank them by the total time spent at the stop. Tag the 1st ranked stop as “Home” and the second as “Work”.

In [ ]:
# 🔹 Vérifier si la table "Recurrent_Stops" existe
check_table_query = text("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_name = 'recurrent_stops';
""")

with engine.connect() as conn:
    result = conn.execute(check_table_query)
    table_exists = result.fetchone() is not None

if not table_exists:
    print("🚨 La table 'Recurrent_Stops' n'existe pas. Création en cours...")

    # 🔹 Création de la table "Recurrent_Stops" si elle n'existe pas
    create_recurrent_stops_query = text("""
        CREATE TABLE Recurrent_Stops AS 
        WITH StopAggregates AS (
            SELECT 
                traj_id,
                ST_Centroid(ST_Collect(geometry)) AS centroid,
                ST_Envelope(ST_Collect(geometry)) AS bbox,
                MIN(t) AS start_time,
                MAX(t) AS end_time,
                (MAX(t) - MIN(t)) AS duration
            FROM mobility_trajectories 
            WHERE segment_type = 'Stop'
            GROUP BY traj_id
        )
        SELECT 
            sa1.traj_id AS stop_id_1,
            sa2.traj_id AS stop_id_2,
            ST_Distance(sa1.centroid, sa2.centroid) AS centroid_distance,
            ST_Intersects(sa1.bbox, sa2.bbox) AS bbox_intersection
        FROM StopAggregates sa1
        JOIN StopAggregates sa2
        ON sa1.traj_id < sa2.traj_id
        WHERE ST_Distance(sa1.centroid, sa2.centroid) < 30
        AND ST_Intersects(sa1.bbox, sa2.bbox) = TRUE;
    """)

    with engine.connect() as conn:
        conn.execute(create_recurrent_stops_query)
        conn.commit()  # 🔹 Valider la création
        print("✅ Table 'Recurrent_Stops' créée avec succès !")

else:
    print("✅ La table 'Recurrent_Stops' existe déjà.")

# 🔹 Vérifier si la table contient des données
check_data_query = text("SELECT COUNT(*) FROM Recurrent_Stops;")

with engine.connect() as conn:
    result = conn.execute(check_data_query)
    row_count = result.scalar()

if row_count == 0:
    print("⚠️ La table 'Recurrent_Stops' est vide. Vérifie les données dans 'mobility_trajectories'.")
else:
    print(f"📊 La table 'Recurrent_Stops' contient {row_count} enregistrements.")

# 🔹 Exécuter la requête de classement des Stops
rank_stops_query = text("""
    WITH StopDurations AS (
        SELECT 
            trajectory_id AS stop_id,
            (MAX(t) - MIN(t)) AS duration  -- Calculer la durée de chaque Stop
        FROM mobility_trajectories
        WHERE segment_type = 'Stop'
        GROUP BY trajectory_id
    ),
    StopRanking AS (
        SELECT 
            rs.stop_id_1 AS stop_id, 
            SUM(sd.duration) AS total_time_spent  -- Calculer la durée totale passée aux stops
        FROM Recurrent_Stops rs
        JOIN StopDurations sd 
        ON rs.stop_id_1 = sd.stop_id
        GROUP BY rs.stop_id_1
    )
    SELECT 
        stop_id,
        total_time_spent,
        CASE 
            WHEN RANK() OVER (ORDER BY total_time_spent DESC) = 1 THEN 'Home'
            WHEN RANK() OVER (ORDER BY total_time_spent DESC) = 2 THEN 'Work'
            ELSE ''
        END AS Label
    FROM StopRanking
    ORDER BY total_time_spent DESC;
""")

# 🔹 Exécuter la requête et charger les résultats dans un DataFrame
with engine.connect() as conn:
    ranked_stops = pd.read_sql(rank_stops_query, conn)

# 🔹 Sauvegarder les résultats dans un fichier CSV
output_path = "Ranked_Stops.csv"
ranked_stops.to_csv(output_path, index=False)

# ✅ Afficher un message de confirmation
print(f"✅ Classement des Stops terminé ! Résultats enregistrés dans '{output_path}'.")

# 13. Rank the “move segments” by the total distance traversed.Rank them by duration. Rank the stops

In [ ]:
rank_moves_query = text("""
    WITH MoveDurations AS (
        SELECT 
            trajectory_id AS move_id,
            SUM(distance_m) AS total_distance,  -- Distance totale en mètres
            (MAX(t) - MIN(t)) AS total_duration -- Durée totale du Move
        FROM mobility_trajectories
        WHERE segment_type = 'Move'
        GROUP BY trajectory_id
    )
    SELECT 
        move_id,
        total_distance,
        total_duration,
        RANK() OVER (ORDER BY total_distance DESC) AS rank_by_distance,
        RANK() OVER (ORDER BY total_duration DESC) AS rank_by_duration
    FROM MoveDurations
    ORDER BY total_distance DESC, total_duration DESC;
""")

# 🔹 Exécuter la requête et charger les résultats dans un DataFrame
with engine.connect() as conn:
    ranked_moves = pd.read_sql(rank_moves_query, conn)

# 🔹 Sauvegarder les résultats dans un fichier CSV
output_path = "Ranked_Moves.csv"
ranked_moves.to_csv(output_path, index=False)

# ✅ Afficher un message de confirmation
print(f"✅ Classement des Move Segments terminé ! Résultats enregistrés dans '{output_path}'.")

In [ ]:
rank_stops_query = text("""
    WITH StopDurations AS (
        SELECT 
            trajectory_id AS stop_id,
            MAX(t) - MIN(t) AS duration,  -- Calculer la durée de chaque visite
            COUNT(*) AS visit_count  -- Nombre de visites au Stop
        FROM mobility_trajectories
        WHERE segment_type = 'Stop'
        GROUP BY trajectory_id
    ),
    StopRanking AS (
        SELECT 
            stop_id,
            visit_count,
            SUM(duration) AS total_time_spent  -- Somme des durées de toutes les visites
        FROM StopDurations
        GROUP BY stop_id, visit_count
    )
    SELECT 
        stop_id,
        visit_count,
        total_time_spent,
        RANK() OVER (ORDER BY visit_count DESC) AS rank_by_frequency,
        RANK() OVER (ORDER BY total_time_spent DESC) AS rank_by_duration
    FROM StopRanking
    ORDER BY total_time_spent DESC, visit_count DESC;
""")
# 🔹 Exécuter la requête et afficher les résultats directement
with engine.connect() as conn:
    ranked_stops = pd.read_sql(rank_stops_query, conn)

# 🔹 Afficher le DataFrame sous forme de tableau
print(ranked_stops)

# 🔹 Visualisation : Histogramme des stops par durée totale
plt.figure(figsize=(10, 5))
plt.bar(ranked_stops["stop_id"].astype(str), ranked_stops["total_time_spent"], color='blue')
plt.xlabel("Stop ID")
plt.ylabel("Total Time Spent (seconds)")
plt.title("Ranking of Stops by Total Time Spent")
plt.xticks(rotation=90)
plt.show()

# 🔹 Visualisation : Histogramme des stops par fréquence de visite
plt.figure(figsize=(10, 5))
plt.bar(ranked_stops["stop_id"].astype(str), ranked_stops["visit_count"], color='green')
plt.xlabel("Stop ID")
plt.ylabel("Visit Count")
plt.title("Ranking of Stops by Frequency of Visits")
plt.xticks(rotation=90)
plt.show()

# 14. Find recurrent trajectory paths : use different methods to identify trajectories that share the same path (e.g., similar traversed distance, distance between start and end location, or between the whole linear shapes, ...)

In [ ]:
query_distances = text("""
    SELECT trajectory_id, SUM(distance_m) AS total_distance 
    FROM mobility_trajectories
    WHERE segment_type = 'Move'
    GROUP BY trajectory_id
    ORDER BY total_distance DESC
    LIMIT 10;
""")

with engine.connect() as conn:
    df_distances = pd.read_sql(query_distances, conn)

print("📏 Distances des 10 plus longs trajets 'Move' :")
print(df_distances)

In [ ]:
query_recurrent_traj = text("""
    WITH TrajectoryFeatures AS (
        SELECT 
            trajectory_id,
            SUM(distance_m) AS total_distance,  
            ST_StartPoint(ST_Collect(geometry)) AS start_point,
            ST_EndPoint(ST_Collect(geometry)) AS end_point
        FROM mobility_trajectories
        WHERE segment_type = 'Move'
        GROUP BY trajectory_id
    )
    SELECT 
        t1.trajectory_id AS traj_1, 
        t2.trajectory_id AS traj_2,
        ABS(t1.total_distance - t2.total_distance) AS distance_diff,  
        ST_Distance(t1.start_point, t2.start_point) AS start_distance,  
        ST_Distance(t1.end_point, t2.end_point) AS end_distance  
    FROM TrajectoryFeatures t1
    JOIN TrajectoryFeatures t2
    ON t1.trajectory_id < t2.trajectory_id
    LIMIT 10;
""")

with engine.connect() as conn:
    df_recurrent = pd.read_sql(query_recurrent_traj, conn)

print("🔍 Exemples de trajets récurrents trouvés :")
print(df_recurrent)

In [ ]:
query_check_moves = text("""
    SELECT trajectory_id, COUNT(*) AS nb_points, SUM(distance_m) AS total_distance
    FROM mobility_trajectories
    WHERE segment_type = 'Move'
    GROUP BY trajectory_id
    ORDER BY nb_points DESC;
""")

with engine.connect() as conn:
    move_stats = pd.read_sql(query_check_moves, conn)

print("📊 Distribution des distances des trajets Move :")
print(move_stats.head(10))

# 🔹 Visualisation de la distribution des distances
plt.figure(figsize=(10, 5))
plt.hist(move_stats["total_distance"].dropna(), bins=30, color='blue', edgecolor='black')
plt.xlabel("Total Distance (meters)")
plt.ylabel("Number of Trajectories")
plt.title("Distribution of Move Trajectories by Distance")
plt.show()

In [ ]:
recurrent_trajectories_query = text("""
    WITH TrajectoryFeatures AS (
        SELECT 
            trajectory_id,
            SUM(distance_m) AS total_distance,  
            ST_StartPoint(ST_Collect(geometry)) AS start_point,  
            ST_EndPoint(ST_Collect(geometry)) AS end_point,  
            ST_Collect(geometry) AS trajectory_shape  
        FROM mobility_trajectories
        WHERE segment_type = 'Move'
        GROUP BY trajectory_id
    )
    SELECT 
        t1.trajectory_id AS traj_1, 
        t2.trajectory_id AS traj_2,
        ABS(t1.total_distance - t2.total_distance) AS distance_diff,  
        ST_Distance(t1.start_point, t2.start_point) AS start_distance,  
        ST_Distance(t1.end_point, t2.end_point) AS end_distance,  
        ST_HausdorffDistance(t1.trajectory_shape, t2.trajectory_shape) AS shape_distance  
    FROM TrajectoryFeatures t1
    JOIN TrajectoryFeatures t2
    ON t1.trajectory_id < t2.trajectory_id
    WHERE 
        ABS(t1.total_distance - t2.total_distance) < 100  
        AND ST_Distance(t1.start_point, t2.start_point) < 50  
        AND ST_Distance(t1.end_point, t2.end_point) < 50  
        AND ST_HausdorffDistance(t1.trajectory_shape, t2.trajectory_shape) < 200;
""")

# 🔹 Exécuter la requête et charger les résultats dans un DataFrame
with engine.connect() as conn:
    recurrent_trajectories = pd.read_sql(recurrent_trajectories_query, conn)

# 🔹 Afficher les résultats
print("🔍 Exemples de trajets récurrents trouvés (version optimisée) :")
print(recurrent_trajectories.head())

# 🔹 Visualisation des différences de distance
plt.figure(figsize=(10, 5))
plt.hist(recurrent_trajectories["distance_diff"].dropna(), bins=20, color='green', edgecolor='black')
plt.xlabel("Difference in Distance (meters)")
plt.ylabel("Number of Similar Trajectories")
plt.title("Distribution of Recurrent Trajectories by Distance")
plt.show()

# 🔹 Visualisation des distances de Hausdorff
plt.figure(figsize=(10, 5))
plt.hist(recurrent_trajectories["shape_distance"].dropna(), bins=20, color='red', edgecolor='black')
plt.xlabel("Hausdorff Distance (meters)")
plt.ylabel("Number of Similar Trajectories")
plt.title("Distribution of Recurrent Trajectories by Shape Similarity")
plt.show()

In [ ]:
# 🔹 Vérification des distances moyennes et des points par trajectoire
query_avg_distance = text("""
    SELECT COUNT(*) AS nb_trajets, AVG(total_distance) AS avg_distance, MIN(total_distance) AS min_distance, MAX(total_distance) AS max_distance
    FROM (
        SELECT trajectory_id, SUM(distance_m) AS total_distance
        FROM mobility_trajectories
        WHERE segment_type = 'Move'
        GROUP BY trajectory_id
    ) AS traj_dist;
""")

with engine.connect() as conn:
    distance_stats = pd.read_sql(query_avg_distance, conn)

print("📊 Statistiques des distances de trajets 'Move' :")
print(distance_stats)

# 15 Let’s consider that the air quality score (AQS) is a function of PM2.5, PM10 and NO2. After normalizing these variables (e.g., by Min-Max feature scaling), the score is obtained by the mean value of the three normalized variables. Compute this temporal score under mobilityDB.

In [ ]:
from sqlalchemy import text
import pandas as pd

# Requête SQL pour calculer AQS en gérant les valeurs NULL
aqs_query = text("""
    WITH MinMaxValues AS (
        SELECT 
            MIN(pm25) AS min_pm25, MAX(pm25) AS max_pm25, AVG(pm25) AS avg_pm25,
            MIN(pm10) AS min_pm10, MAX(pm10) AS max_pm10, AVG(pm10) AS avg_pm10,
            MIN(no2) AS min_no2, MAX(no2) AS max_no2, AVG(no2) AS avg_no2
        FROM sensor_measures
    ),
    NormalizedData AS (
        SELECT 
            sm.time,
            (COALESCE(sm.pm25, mm.avg_pm25) - mm.min_pm25) / NULLIF(mm.max_pm25 - mm.min_pm25, 0) AS pm25_norm,
            (COALESCE(sm.pm10, mm.avg_pm10) - mm.min_pm10) / NULLIF(mm.max_pm10 - mm.min_pm10, 0) AS pm10_norm,
            (COALESCE(sm.no2, mm.avg_no2) - mm.min_no2) / NULLIF(mm.max_no2 - mm.min_no2, 0) AS no2_norm
        FROM sensor_measures sm, MinMaxValues mm
    )
    SELECT 
        time,
        (pm25_norm + pm10_norm + no2_norm) / 3 AS air_quality_score
    FROM NormalizedData
    ORDER BY time;
""")

# 🔹 Exécuter la requête et charger les résultats dans un DataFrame
with engine.connect() as conn:
    aqs_df = pd.read_sql(aqs_query, conn)

# 🔹 Afficher les premiers résultats
print("\n✅ Calcul de l'Air Quality Score (AQS) terminé !\n")
print(aqs_df.head(10))  # Affiche les 10 premières lignes

# Optionnel : Sauvegarder les résultats dans un fichier CSV
aqs_df.to_csv("Air_Quality_Score.csv", index=False)
print("\n📂 Résultats enregistrés dans 'Air_Quality_Score.csv'\n")

# 16. Rank the days per ascending cumulated AQS

In [ ]:
rank_days_query = text("""
    WITH DailyAQS AS (
        SELECT 
            DATE(time) AS day,
            SUM(air_quality_score) AS total_aqs
        FROM Air_Quality_Score
        GROUP BY DATE(time)
    )
    SELECT 
        day,
        total_aqs,
        RANK() OVER (ORDER BY total_aqs ASC) AS rank
    FROM DailyAQS
    ORDER BY total_aqs ASC;
""")

with engine.connect() as conn:
    ranked_days_df = pd.read_sql(rank_days_query, conn)

print("✅ Classement des jours par AQS cumulé terminé !")
print(ranked_days_df.head())  # Afficher un aperçu

# 📂 Sauvegarde en CSV
output_path = "Ranked_Days_AQS.csv"
ranked_days_df.to_csv(output_path, index=False)
print(f"📂 Résultats enregistrés dans '{output_path}'.")

# 17. Compare the cumulated AQS , the min ,max, and average between days and nights ,and between the stops and the moves periods.

In [ ]:
compare_aqs_query = text("""
    WITH AggregatedAQS AS (
        SELECT 
            period,
            segment,
            SUM(air_quality_score) AS total_aqs,
            MIN(air_quality_score) AS min_aqs,
            MAX(air_quality_score) AS max_aqs,
            AVG(air_quality_score) AS avg_aqs
        FROM Air_Quality_Score
        GROUP BY period, segment
    )
    SELECT * FROM AggregatedAQS;
""")

with engine.connect() as conn:
    comparison_df = pd.read_sql(compare_aqs_query, conn)

print("✅ Comparaison des scores AQS entre les périodes terminée !")
print(comparison_df)

# 📂 Sauvegarde en CSV pour analyse
output_path = "AQS_Comparison.csv"
comparison_df.to_csv(output_path, index=False)
print(f"📂 Résultats enregistrés dans '{output_path}'.")